<a href="https://colab.research.google.com/github/mvdheram/AHCVS_Notes/blob/main/07_Heirarchical_Occlusion_Maps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hierarchical Occlusion Maps 

Source : https://dl.acm.org/doi/pdf/10.1145/258734.258781?casa_token=yfHnfjHa304AAAAA:O6A0DezAU1zs6d2GIbNPzW6XSaDzixmUOeUt7puubSqBpSBU27iy_TRAocEQy86hW5JGWuCrJUc0sOs

**Why?**
* **Visibility culling algorithm dealing with small occluders to achieves significant speedup in interactive walkthroughs of models with high depth complexity.**
  * Algorithm for Dealing with small occluders
    * Aggregated occluders : Composition of many small occluders aggregate to large occluders based on viewpoint 
    * Uses :
      * Occluder fusion  (Union set of occluders)

**What?**
* HOM : **Records the aggregate projection of occluders** onto the image plane
 **at different resolutions**.
  * **Hierarchical maps capture the cumulative occluding effects of the occluders**

**How?**
  * 2 Tests for visibility problem (occlusion culling)
    * Overlap test
      * Test potential occludee lies within (overlaps) the screen porjection  of aggregated occluders.
      * Uses : 
        * Occlusion maps
    * Depth test
      * Test potential occludee lies beind the occluders 
      * Uses :
        * Depth estimation buffer

## Modified Rendring pipeline 

**2 new stages in rendering piepline**

  * For reach frame :
    1.  Construction of the Occlusion Map Hierarchy
    2. Visibility Culling with Hierarchical Occlusion Maps and rendering 

### Construction of Occlusion Map Hierarchy 

Pipeline consists of 5 stages :

1. **Occluder database** :
    * Computed in preprocessing stage from the model database
      * Why ?
        * Temporal coherence Selecting occluders from the previous frame.
    * Occluders stored in bounding volume hierarchy
      * What is bounding volume hierarchy ?
        * Spatial data structure similar to BSP trees for clustering objects hierarchically using sphere or boxes (tree structure)
    * All objects of 3D scene are examined and occluder objects selected when 
      * Objects not below certain size (small objects)
      * Objects which are not too complex 

2. View frustum culling 
  * The algorithm traverses the bounding volume hierarchy of the occluder database to find occluders lying in the viewing frustum.

3. Occluder selection  
  * The algorithm selects a subset of the
occluders lying in the viewing frustum.
  * How?
    * Sort all found occluders by distance and starting from the camera position, choose successive occluders
until a maximum value of polygon complexity (polygon count)/limit is reached.
    * It utilizes temporal
coherence between successive frames.  

4.  Occluder rendering 
  * Selected occluders are rendered as black and white image in frame buffer, where white pixel represents occluded objects.
  * A depth estimation buffer is built to record
the depth of the occluders.

5. Computation of occlusion map hierarchy
  * After rendering the occluders into black and white image, the pixel are recursively coarsened by combining and applying mean filter for blocks of pixel. 


### Visibility culling and rendering 

Pipeline consists of 3 stages :

Traverse the bounding volume hierarchy of  model database
  1. View frustum culling 
    * Apply view frustum culling to the model database.
  2. Depth comparison
    * Potential occludee, are conservatively checked whether it is behind occluders.
  3. Overlap test with occlusion maps
    * Traverse occlusion map hierarchy from low resolutiton to high resolution and check if 2D projection of occludee overlaps the "opaque/white pixel" part of occlusion map.  


#### Occlusion map

**Opacity** 
  * **Ratio of colored pixel to total number of pixel** 
  * How ?
    * Project the occluder objects into 2D image space using frame buffer
    * Some pixels are colored while other are colorless (blank/white)

**Occlusion map** 
* **An occlusion map is a two-dimensional
array in which each pixel records the opacity of a rectangular block
of screen space.** 
* Any rendered image can have an occlusion map with opacity values stored per pixel. 

**Computation of occlusion map**
  * **Render occluders as complete white objects in frame buffer. This forms occlusion map with highest resolution**
  * Hierarchical occlusion map is formed by aggregation of multiple occluders in the 2D imgae space.
    * How?
      * Merge 2x2 pixel values of highest resolution (lower layer) recursively (by taking mean) to higher levels (coarser resolution).
      * **Improvement of hierarchy construction** 
        * Idea : **Occlusion map of 2N * 2N to N*N using texture memory**
        * How ?
          * Render occlusion map of highest resolution 2N*2N into frame buffer
          * Load the rendered image into texture memory

**Visibility culling using occlusion map**
  1. **Overlap test with occlusion maps**
    * How?
      1. The corners of the 3D boundingbox of object is projected into image space -> 2D bounding box(image space)
      2. Use HOM and **determine the *level* of hierarchy** where the size of pixel (HOM) matches size of  2D boundingbox. 
        * At that level **check** if **intersected pixel in HOM completely overlap.**
        * If not, go deeper into hierarchy and check the subpixels intersection.
        * If all the intersected pixel in HOM corresponds to opaque(white pixel in HOM) then the object is completely inside of an aggregated occluder.
  2. **Depth test** 
    * Why ?
      * Determine the position of ocludee (behind occluders) after overlap test.
    * How ?
      1. Variant 1 (single z-value test)
        * Store z-plane (farthes z-value) of aggregated occluders
      2. Variant 2 (multiple z-value test using depth buffer)
        * Create a depth buffer (2D ) with individual z-value of all occluders
        * Project object to be tested into the 2D image space
        * Compute the closest z value
        * Check against depth buffer

**Approximate visibility culling**

* If the scene consists of aggregated occluders where there are some small haps/holes.
  * Treat these holes as of they were closed by occluders
  * Thus not render the objects visible through these holes
* **Why ?**
  * Accelerate the rendering speed 


 